In [ ]:
#https://www.kaggle.com/code/akshitrai/chatbot-jarvis/notebook



In [1]:
import numpy as np 
import string
from nltk.corpus import stopwords
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('/Users/lana-n/datacsv/dialogs.txt',sep='\t')

In [3]:
df

,"hi, how are you doing?",i'm fine. how about yourself?
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?
...,...,...
3719,that's a good question. maybe it's not old age.,are you right-handed?
3720,are you right-handed?,yes. all my life.
3721,yes. all my life.,you're wearing out your right hand. stop using...
3722,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [4]:
a = pd.Series(df.columns)

In [5]:
df

,"hi, how are you doing?",i'm fine. how about yourself?
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?
...,...,...
3719,that's a good question. maybe it's not old age.,are you right-handed?
3720,are you right-handed?,yes. all my life.
3721,yes. all my life.,you're wearing out your right hand. stop using...
3722,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [7]:
a = a.rename({0: df.columns[0],1: df.columns[1]})

In [9]:
a

hi, how are you doing?                  hi, how are you doing?
i'm fine. how about yourself?    i'm fine. how about yourself?
dtype: object

In [10]:
#added some common questions:

b = {'Questions':'Hi','Answers':'hello'}

c = {'Questions':'Hello','Answers':'hi'}

d= {'Questions':'how are you','Answers':"i'm fine. how about yourself?"}

e= {'Questions':'how are you doing','Answers':"i'm fine. how about yourself?"}


In [11]:
df = df.append(a,ignore_index=True)

<ipython-input-11-8a35d942b5da>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a,ignore_index=True)


In [12]:
df.columns=['Questions','Answers']

In [13]:
df = df.append([b,c,d,e],ignore_index=True)

<ipython-input-13-cb63184fe63a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([b,c,d,e],ignore_index=True)


In [14]:
df

,Questions,Answers
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?
...,...,...
3724,"hi, how are you doing?",i'm fine. how about yourself?
3725,Hi,hello
3726,Hello,hi
3727,how are you,i'm fine. how about yourself?


In [15]:
df = df.append(c,ignore_index=True)
df = df.append(d,ignore_index=True)
df = df.append(d,ignore_index=True)

<ipython-input-15-394fdd70ffc2>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(c,ignore_index=True)
<ipython-input-15-394fdd70ffc2>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,ignore_index=True)
<ipython-input-15-394fdd70ffc2>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,ignore_index=True)


In [16]:
df

,Questions,Answers
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?
...,...,...
3727,how are you,i'm fine. how about yourself?
3728,how are you doing,i'm fine. how about yourself?
3729,Hello,hi
3730,how are you,i'm fine. how about yourself?


In [17]:
def cleaner(x):
    return [a for a in (''.join([a for a in x if a not in string.punctuation])).lower().split()]

In [19]:
# Decision Tree Classifier:

Pipe = Pipeline([
    ('bow',CountVectorizer(analyzer=cleaner)),
    ('tfidf',TfidfTransformer()),
    ('classifier',DecisionTreeClassifier())
])

In [20]:
Pipe.fit(df['Questions'],df['Answers'])

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function cleaner at 0x7f9fa39601f0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', DecisionTreeClassifier())])

In [21]:
#now we can try to talk to our chat bot:

Pipe.predict(['hi'])[0]

'hello'

In [22]:
Pipe.predict(['how are you'])[0]

"i'm fine. how about yourself?"

In [23]:
Pipe.predict(['great'])[0]

'i appreciate that.'

In [24]:
Pipe.predict(['What are you doing'])[0]

"i'm going to change the light bulb. it burnt out."

In [25]:
Pipe.predict(['What about the movies'])[0]

'what do you mean?'

In [26]:
Pipe.predict(['Lets go to the movies'])[0]

"that's a great idea."

In [27]:
Pipe.predict(['What movie would you like to see?'])[0]

"oh, i don't care. you're the one who wants to go out."